In [1]:
import pandas as pd
import numpy as np
# 用於二元目標變數
from scipy.stats import pointbiserialr

In [2]:
# Import GDP Data
from data import OverallGeneralEcon, df, MacroInfo, JuridicalInfo

c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Dream_report = pd.read_sql(qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stock_info = pd.read_sql(stock_qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MacroInfo = pd.read_sql(macro_qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:106: UserWarning: pandas 

In [6]:
# 名目、實質、時間
"""
MacroInfo
名目有效匯率指數–台灣
台灣消費者信心指數
台灣消費者信心指數–未來半年國內經濟景氣
台灣消費者信心指數–未來半年國內物價水準
美國消費者信心指數
經濟成長率(GDP)–全年
--
實質國內生產毛額(GDP)–台幣 -> 只更新到202009
實質GDP成長率(台灣) -> 只更新到202212
--
貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均
臺灣製造業PMI - 新增出口訂單 - 基礎原物料
台灣外銷訂單金額–全球
消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數
美國核心PPI、美國核心PPI月增率
進口物價指數、出口物價指數年增率

sysallunTWESPF
外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)
景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)

sysTaiStaValMAC
景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)

其他
財報、營收
"""

'\n名目有效匯率指數–台灣\n台灣消費者信心指數\n台灣消費者信心指數–未來半年國內經濟景氣\n台灣消費者信心指數–未來半年國內物價水準\n美國消費者信心指數\n經濟成長率(GDP)–全年\n實質國內生產毛額(GDP)–台幣\n實質GDP成長率(台灣)\n貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均\n臺灣製造業PMI - 新增出口訂單 - 基礎原物料\n台灣外銷訂單金額–全球\n消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數\n美國核心PPI、美國核心PPI月增率\n進口物價指數、出口物價指數年增率\n'

In [18]:
# Improt data
# 名目有效匯率指數
NominalExchageRate = MacroInfo[MacroInfo['名稱']=='名目有效匯率指數–台灣'][['年月','數值','名稱']]
display(NominalExchageRate.head(3))

# 台灣消費者信心指數
TW_ICS = MacroInfo[MacroInfo['名稱']=='台灣消費者信心指數'][['年月','數值','名稱']]
display(TW_ICS.head(3))

# 消費者物價指數(CPI)年增率
TW_CPIGrowth = MacroInfo[MacroInfo['名稱']=='消費者物價指數(CPI)年增率'][['年月','數值','名稱']]
display(TW_CPIGrowth.head(3))

# 經濟成長率(GDP)–全年
TW_GDPGrowth = MacroInfo[MacroInfo['名稱']=='經濟成長率(GDP)–單季'][['年月','數值','名稱']]
display(TW_GDPGrowth.head(3))

,年月,數值,名稱
1848,202311,101.05,名目有效匯率指數–台灣
3064,202310,100.99,名目有效匯率指數–台灣
4865,202309,101.07,名目有效匯率指數–台灣


,年月,數值,名稱
1042,202312,70.06,台灣消費者信心指數
2110,202311,69.40,台灣消費者信心指數
3327,202310,69.06,台灣消費者信心指數


,年月,數值,名稱
309,202312,2.71,消費者物價指數(CPI)年增率
1170,202311,2.90,消費者物價指數(CPI)年增率
2324,202310,3.05,消費者物價指數(CPI)年增率


,年月,數值,名稱
0,202412,1.06,經濟成長率(GDP)–單季
27,202409,2.12,經濟成長率(GDP)–單季
42,202406,4.77,經濟成長率(GDP)–單季


In [20]:
JuridicalInfo.columns
#上市買賣超金額(百萬)、上櫃買賣超金額(百萬)、合計買進金額(百萬)

Index(['日期', '代號', '名稱', '上市買進金額(百萬)', '上市賣出金額(百萬)', '上市買賣超金額(百萬)',
       '上櫃買進金額(百萬)', '上櫃賣出金額(百萬)', '上櫃買賣超金額(百萬)', '合計買進金額(百萬)', '合計賣出金額(百萬)',
       '合計買賣超金額(百萬)', '上櫃股票買進金額(百萬)', '上櫃股票賣出金額(百萬)', '上櫃股票買賣超金額(百萬)',
       '興櫃買進金額(百萬)', '興櫃賣出金額(百萬)', '興櫃買賣超金額(百萬)', 'RTIME'],
      dtype='object')

In [16]:
NominalGDP = OverallGeneralEcon[['time','Nominal_GDP']]
NominalGDP = NominalGDP.rename(columns={'time':'quarter'})
#print(NominalGDP)
#print(type(NominalGDP))

In [17]:
# 將df的日期轉換為季度
df_copy = df.copy()
df_copy['quarter'] = (pd.to_datetime(df_copy['ddate'], format='%Y%m').dt.year.astype(str) + 'Q' + pd.to_datetime(df_copy['ddate'], format='%Y%m').dt.quarter.astype(str))

# 使用merge方法合併兩個data frame
df_copy = pd.merge(df_copy, NominalGDP, on='quarter', how='left').dropna()

In [19]:
df_copy.columns

Index(['ddate', 'stockid', 'direction', 'rev', 'mom', 'yoy', 'revd1', 'yoyd1',
       'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
       'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
       'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
       'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
       'revd12', 'yoyd12', 'mond12', 'quarter', 'Nominal_GDP'],
      dtype='object')

In [21]:
# variable and target columns
target_col_GDP =  ['direction']
variable_col_GDP = ['revd1', 'yoyd1',
                         'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
                         'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
                         'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
                         'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
                         'revd12', 'yoyd12', 'mond12','Nominal_GDP' ]

y_GDP = df_copy[target_col_GDP] # overall y
X_GDP = df_copy[variable_col_GDP] # overall X

In [27]:
# 計算變數相關性，用以決定變數
correlation, _ = pointbiserialr(df_copy['direction'], df_copy['Nominal_GDP'])

print(correlation)

correlation2, _ = pointbiserialr(df_copy['direction'], df_copy['revd1'])
print(correlation2)

0.009671734212467519
-0.0023818022480477617


In [44]:
# 計算每個變數與目標變數的點二序列相關係數，並儲存到一個字典中
correlations = {}
for variable in variable_col_GDP:
    if variable != 'direction':
        correlation, _ = pointbiserialr(df_copy['direction'], df_copy[variable])
        correlation = round(correlation, 4)
        correlations[variable] = correlation
        #print(f'Correlation between target and {variable}: {correlation}')

#print(correlations)

# 找出相關性最大和最小的變數
max_corr_var = max(correlations, key=correlations.get)
min_corr_var = min(correlations, key=correlations.get)
print(max_corr_var)
print(f'Variable with highest correlation: {max_corr_var}, correlation: {correlations[max_corr_var]}')
print(f'Variable with lowest correlation: {min_corr_var}, correlation: {correlations[min_corr_var]}')


revd12
Variable with highest correlation: revd12, correlation: 0.0105
Variable with lowest correlation: yoyd1, correlation: -0.0105
